In [1]:
import os
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import numpy as np
import cv2

# Define a combined function for applying blur and brightness adjustment with kernel size and alpha as parameters
def blur_and_brightness(image, kernel_size=(15, 15), alpha=0.7, beta=3):
    """
    Apply Gaussian blur and brightness adjustment to an image.
    
    Parameters:
    image (ndarray): The input image.
    kernel_size (tuple): The size of the Gaussian blur kernel (must be odd).
    alpha (float): The factor for brightness adjustment (lower values result in darker images).
    beta (int): The amount to be added to each pixel's brightness (positive for brighter, negative for darker).
    
    Returns:
    ndarray: The processed image with blur and brightness adjustment.
    """
    # Convert image from RGB to BGR format for OpenCV
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    # Apply Gaussian blur with the specified kernel size (ensure it's odd)
    image = cv2.GaussianBlur(image, kernel_size, 0)
    
    # Adjust brightness by scaling pixel values using the specified alpha and adding beta
    image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    
    # Convert the image back to RGB for TensorFlow or saving
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    return image

# Augmentation settings for the first stage (rotation, shift, etc.)
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=False,
    fill_mode='nearest'
)

def processaugment(base_dir, output_dir):
    # Step 1: Augment images from base_dir and save them to output_dir
    for folder in os.listdir(base_dir):
        folder_path = os.path.join(base_dir, folder)
        
        if os.path.isdir(folder_path):
            print(f'Processing folder: {folder}')
            
            # Create corresponding output directory if it doesn't exist
            output_folder = os.path.join(output_dir, folder)
            if not os.path.exists(output_folder):
                os.makedirs(output_folder)
            
            for image_file in os.listdir(folder_path):
                img_path = os.path.join(folder_path, image_file)
                
                if img_path.endswith(('.png', '.jpg', '.jpeg')):  # Filter image files
                    
                    # Copy the original image to the output folder
                    shutil.copy(img_path, output_folder)
    
                    # Load the original image
                    img = load_img(img_path)
                    x = img_to_array(img)
                    x = np.expand_dims(x, axis=0)
                    
                    # Extract the base name without extension
                    base_name, ext = os.path.splitext(image_file)
                    
                    # Augment the original image and save in output_dir
                    i = 0
                    for batch in datagen.flow(x, batch_size=1, save_to_dir=output_folder, save_prefix=f'{base_name}_aug', save_format='jpeg'):
                        i += 1
                        if i >= 5:  # Generate 5 augmented images per original image
                            break
        #break
    # Step 2: Apply blur and brightness adjustments with custom kernel and alpha to both original and augmented images in output_dir
    for folder in os.listdir(output_dir):
        folder_path = os.path.join(output_dir, folder)

        if os.path.isdir(folder_path):
            print(f'Re-processing folder: {folder}')
            
            for image_file in os.listdir(folder_path):
                img_path = os.path.join(folder_path, image_file)

                if img_path.endswith(('.png', '.jpg', '.jpeg')):  # Filter image files
                    
                    # Load the image (original or augmented) for processing
                    img = load_img(img_path)
                    img_array = img_to_array(img)
                    
                    # Step 2.1: Apply heavy blur and brightness adjustments
                    heavy_blurred_img = blur_and_brightness(img_array, kernel_size=(25, 25), alpha=0.5, beta=3)
                    base_name, ext = os.path.splitext(image_file)
                    save_path_heavy = os.path.join(folder_path, f'{base_name}_heavy_blur_brightness{ext}')
                    cv2.imwrite(save_path_heavy, cv2.cvtColor(heavy_blurred_img, cv2.COLOR_RGB2BGR))
                    
                    # Step 2.2: Apply medium blur and brightness adjustments
                    medium_blurred_img = blur_and_brightness(img_array, kernel_size=(21, 21), alpha=0.6, beta=3)
                    save_path_medium = os.path.join(folder_path, f'{base_name}_medium_blur_brightness{ext}')
                    cv2.imwrite(save_path_medium, cv2.cvtColor(medium_blurred_img, cv2.COLOR_RGB2BGR))
                    
                    # Step 2.3: Apply light blur and brightness adjustments
                    light_blurred_img = blur_and_brightness(img_array, kernel_size=(11, 11), alpha=0.8, beta=3)
                    save_path_light = os.path.join(folder_path, f'{base_name}_light_blur_brightness{ext}')
                    cv2.imwrite(save_path_light, cv2.cvtColor(light_blurred_img, cv2.COLOR_RGB2BGR))
                    
                    # Step 2.4: Apply semi-heavy blur and moderate brightness adjustments
                    semi_blurred_img = blur_and_brightness(img_array, kernel_size=(15, 15), alpha=0.7, beta=3)
                    save_path_semi = os.path.join(folder_path, f'{base_name}_semi_blur_brightness{ext}')
                    cv2.imwrite(save_path_semi, cv2.cvtColor(semi_blurred_img, cv2.COLOR_RGB2BGR))


    print('Augmentation and re-augmentation (with custom kernel sizes and alpha values) completed!')



# Path to the directory you want to remove
directory_to_remove = r"C:\Users\Multimatics\plat_nomor\textplataug\tensor"

# Check if the directory exists, then remove it
if os.path.exists(directory_to_remove):
    shutil.rmtree(directory_to_remove)
    print(f"Directory {directory_to_remove} has been removed successfully.")
else:
    print(f"Directory {directory_to_remove} does not exist.")


# Define the directories and the augmentation for each image set (huruf and angka)

print("=============Huruf==========")
base_dir = r'textplat\tensor\huruf\train'  # Original directory for images
output_dir = r'textplataug\tensor\huruf\train'  # Directory to store augmented images

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
processaugment(base_dir, output_dir)


base_dir = r'textplat\tensor\huruf\test'  # Original directory for images
output_dir = r'textplataug\tensor\huruf\test'  # Directory to store augmented images

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
processaugment(base_dir, output_dir)

print("=============Angka==========")

# Define the directories and the augmentation for each image set (huruf and angka)
base_dir = r'textplat\tensor\angka\train'  # Original directory for images
output_dir = r'textplataug\tensor\angka\train'  # Directory to store augmented images

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
processaugment(base_dir, output_dir)

base_dir = r'textplat\tensor\angka\test'  # Original directory for images
output_dir = r'textplataug\tensor\angka\test'  # Directory to store augmented images

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
processaugment(base_dir, output_dir)


Directory C:\Users\Multimatics\plat_nomor\textplataug\tensor has been removed successfully.
=============Huruf==========
Processing folder: A
Processing folder: B
Processing folder: C
Processing folder: D
Processing folder: E
Processing folder: F
Processing folder: G
Processing folder: H
Processing folder: I
Processing folder: J
Processing folder: K
Processing folder: L
Processing folder: M
Processing folder: N
Processing folder: O
Processing folder: P
Processing folder: Q
Processing folder: R
Processing folder: S
Processing folder: T
Processing folder: U
Processing folder: V
Processing folder: W
Processing folder: X
Processing folder: Y
Processing folder: Z
Processing folder: ~
Re-processing folder: A
Re-processing folder: B
Re-processing folder: C
Re-processing folder: D
Re-processing folder: E
Re-processing folder: F
Re-processing folder: G
Re-processing folder: H
Re-processing folder: I
Re-processing folder: J
Re-processing folder: K
Re-processing folder: L
Re-processing folder: M


In [2]:
import os
from PIL import Image

# Function to check if files in a directory are valid images and delete if corrupted
def check_and_delete_corrupted_images(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                with Image.open(file_path) as img:
                    img.verify()  # Verify that it is an image
            except (IOError, SyntaxError) as e:
                print(f"Deleting corrupted or non-image file: {file_path}")
                os.remove(file_path)  # Delete the corrupted or non-image file

# Define your dataset directories for checking
train_dir = r"C:\Users\Multimatics\plat_nomor\textplataug\tensor\angka\train"
test_dir = r"C:\Users\Multimatics\plat_nomor\textplataug\tensor\angka\test"
huruf_train_dir = r"C:\Users\Multimatics\plat_nomor\textplataug\tensor\huruf\train"
huruf_test_dir = r"C:\Users\Multimatics\plat_nomor\textplataug\tensor\huruf\test"

# Run the check and delete for each directory
check_and_delete_corrupted_images(train_dir)
check_and_delete_corrupted_images(test_dir)
check_and_delete_corrupted_images(huruf_train_dir)
check_and_delete_corrupted_images(huruf_test_dir)

print("Completed checking and deleting corrupted/non-image files.")


Completed checking and deleting corrupted/non-image files.
